In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
!pip install flask-ngrok


In [13]:
pip install -U flask-cors

In [14]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request, redirect
from flask import jsonify
from flask_cors import CORS, cross_origin

In [15]:
import cv2
from keras.models import load_model
import tensorflow as tf
import numpy as np
from PIL import Image

Using TensorFlow backend.


In [16]:
app = Flask(__name__)
#cors = CORS(app, resources={r"/foo": {"origins": "file://"}})
#app.config['CORS_HEADERS'] = 'Content-Type'
CORS(app)
run_with_ngrok(app)

In [17]:
model=tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/ml/trained-model.h5',compile=False)

In [18]:
def prepare(image):
    IMG_SIZE=30
    new_array= cv2.resize(np.float32(image),(IMG_SIZE,IMG_SIZE))
    return new_array.reshape(1,IMG_SIZE,IMG_SIZE,3)

In [19]:
def detail(classid):
  switcher={
      0:'Speed limit (20km/h)',
      1:'Speed limit (30km/h)',
      2:'Speed limit (50km/h)',
      3:'Speed limit (60km/h)',
      4:'Speed limit (70km/h)',
      5:'Speed limit (80km/h)',
      6:'End of speed limit (80km/h)',
      7:'Speed limit (100km/h)',
      8:'Speed limit (120km/h)',
      9:'No passing',
      10:'No passing for vechiles over 3.5 metric tons',
      11:'Right-of-way at the next intersection',
      12:'Priority road',
      13:'Yield',
      14:'Stop',
      15:'No vechiles',
      16:'Vechiles over 3.5 metric tons prohibited',
      17:'No entry',
      18:'General caution',
      19:'Dangerous curve to the left',
      20:'Dangerous curve to the right',
      21:'Double curve',
      22:'Bumpy road',
      23:'Slippery road',
      24:'Road narrows on the right',
      25:'Road work',
      26:'Traffic signals',
      27:'Pedestrians',
      28:'Children crossing',
      29:'Bicycles crossing',
      30:'Beware of ice/snow',
      31:'Wild animals crossing',
      32:'End of all speed and passing limits',
      33:'Turn right ahead',
      34:'Turn left ahead',
      35:'Ahead only',
      36:'Go straight or right',
      37:'Go straight or left',
      38:'Keep right',
      39:'Keep left',
      40:'Roundabout mandatory',
      41:'End of no passing',
      42:'End of no passing by vechiles over 3.5 metric tons'
  }
  return switcher.get(classid,"Invalid")

In [10]:
detail(0)

'Speed limit (20km/h)'

In [20]:
@app.route("/predict", methods=['GET','POST'])
#@cross_origin(origin='*',headers=['Content-Type','Authorization'])
def predict():
    data =request.files["inpFile"]
    image = Image.open(request.files["inpFile"])
    image = np.array(image)
    predict=model.predict([prepare(image)])
    #pred_name = CATEGORIES[np.argmax(predict)]
    pred_name =  detail(np.argmax(predict))   
    response = {'finalResult' : pred_name}
    print(response) 
    return jsonify(response)

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2042f3d07195.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2020-07-09 16:01:21,872] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_reque

{'finalResult': 'Turn left ahead'}


127.0.0.1 - - [09/Jul/2020 16:05:06] "POST /predict HTTP/1.1" 200 -


{'finalResult': 'Turn left ahead'}
